# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.14.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

key = "Heart-Failure"

if key in ws.datasets.keys(): 
        dataset = ws.datasets[key] 

# choose a name for experiment
experiment_name = 'Heart-Fail-Pred'

experiment=Experiment(ws, experiment_name)

In [3]:

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [4]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 28,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 5,
    "featurization": 'auto'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(task = "classification",
                             training_data=dataset,
                             compute_target=cpu_cluster,
                             label_column_name="DEATH_EVENT",   
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [5]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote or ADB.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: Heart-Fail-Pred,
Id: AutoML_7ca38952-486b-4c1a-8038-23ca75587cdd_91,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               random_state=0,
                                                                                               reg_alpha=2.1875,
                                       

In [12]:
#TODO: Save the best model
import joblib

joblib.dump(value=fitted_model, filename="best-trained-model.pkl")

['best-trained-model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [34]:
from azureml.core.model import Model
from azureml.core.model import InferenceConfig
#from azureml.core.webservice import AciWebservice
from azureml.core import Workspace, Environment
from azureml.core.webservice import LocalWebservice
from azureml.core.conda_dependencies import CondaDependencies


model = Model.register(model_path="best-trained-model.pkl",
                       model_name="best-trained-model",
                       workspace = ws)



service_name = 'my-custom-env-service'

env = Environment.get(workspace=ws, name="AzureML-AutoML")
#conda_dep = CondaDependencies()

#conda_dep.add_conda_package("xgboost")

#env.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script='score.py', environment=env)
#aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, enable_app_insights=True)
deployment_conf = LocalWebservice.deploy_configuration(port=6789)


service = Model.deploy(workspace=ws,
                          name=service_name,
                          models=[model],
                          inference_config=inference_config,
                          #deployment_config=aci_config)
                          deployment_config=deployment_conf)
                       
service.wait_for_deployment(show_output=True)


print(service.state)
print("scoring URI: " + service.scoring_uri)

Registering model best-trained-model
Generating Docker build context.
Package creation Succeeded
Logging into Docker registry viennaglobal.azurecr.io
Logging into Docker registry viennaglobal.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM viennaglobal.azurecr.io/azureml/azureml_ec19bfb89fc3cf9052877a75bea2cf97
 ---> 15833e00453f
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> accbda2a995b
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjhhNTZlYmM1LWNhYTctNGE0My1iOGVkLTFjMTg5NWU0NjA1MSIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTExOTc1MCIsImFjY291bnROYW1lIjoiYXp1cmUtbWwtd3MiLCJ3b3Jrc3BhY2VJZCI6ImFjZTQ2NDA1LTM0NzUtNDU0OS1hZWM5LTYxYjY2MjhhYzIwNiJ9LCJtb2RlbHMiOnt9LCJtb2RlbHNJbmZvIjp7fX0= | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 12e8a5a36da6
 ---> 147812f36c14
Step 4/5 : RUN mv '/var/azureml-app/tmpfgol7pm_.py' /var/azureml-app/main.py
 ---> Running in ed94c2301d72
 ---> c3b

In [29]:
service.reload()

Container has been successfully cleaned up.
Starting Docker container...
Docker container running.


TODO: In the cell below, send a request to the web service you deployed to test it.

In [46]:
import requests
import json

# URL for the web service
scoring_uri = service.scoring_uri

headers = {'Content-Type':'application/json'}

# If the service is authenticated, set the key or token
#key = '<your key or token>'

# Two sets of data to score, so we get two results back
data = {
#        "data": [{
#                 
#                     "age": 75, 
#                     "anaemia": 0, 
#                     "creatinine_phosphokinase": 582, 
#                     "diabetes": 0, 
#                     "ejection_fraction": 20, 
#                     "high_blood_pressure": 1, 
#                     "platelets": 265000, 
#                     "serum_creatinine": 1.9, 
#                     "serum_sodium": 130, 
#                     "sex": 1, 
#                     "smoking": 0,
#                     "time": 4
#        }]
        "data": [{
                 
                     75, 
                     0, 
                     582, 
                     0, 
                     20, 
                     1, 
                     265000, 
                     1.9, 
                     130, 
                     1, 
                     0,
                     4
        }]
                
        }
         
# Convert to JSON string
input_data = json.dumps(data)
print(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.status_code)
print(resp.text)

TypeError: Object of type 'set' is not JSON serializable

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)

In [ ]:
# Deleting the deployed model service

service.delete(delete_cache=True, delete_image=False, delete_volume=True)